In [ ]:
#Upload the files
from google.colab import files
uploaded = files.upload()

In [ ]:
#LSTM
import numpy as np
from math import sqrt
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

rmse2 = 0

tr_filename="train_NREL_solar_data.csv"
train_data = np.loadtxt(tr_filename,delimiter=',')

va_filename="validate_NREL_solar_data.csv"
validate_data = np.loadtxt(va_filename,delimiter=',')

te_filename="test_NREL_solar_data.csv"
test_data = np.loadtxt(te_filename,delimiter=',')

x_tr  = train_data[:,0:9]
t_tr  = train_data[:,-1]

x_va  = validate_data[:,0:9]
t_va  = validate_data[:,-1]

x_te  = test_data[:,0:9]
t_te  = test_data[:,-1]

Ndays_tr = x_tr.shape[0]//11
Ndays_va = x_va.shape[0]//11
Ndays_te = x_te.shape[0]//11

train_x = x_tr.reshape(Ndays_tr,11,9)
train_t = t_tr.reshape(Ndays_tr,11,1)

validate_x = x_va.reshape(Ndays_va,11,9)
validate_t = t_va.reshape(Ndays_va,11,1)

test_x = x_te.reshape(Ndays_te,11,9)
test_t = t_te.reshape(Ndays_te,11,1)

model = Sequential()
    #model.add(LSTM(30,input_shape=(11,9)))
    #model.add(Dense(9,activation='linear'))
    #model.add(LSTM(input_dim=9,output_dim=25,return_sequences=True))
model.add(LSTM(50,input_shape=(11,9),return_sequences=True))
model.add(Dense(1,activation='linear'))
model.compile(loss='mse',optimizer='adam')
    #history = model.fit(train_x,train_t,epochs=50,validation_data=(test_x,test_t))
history = model.fit(train_x,train_t,epochs=100,batch_size=50,validation_data=(validate_x,validate_t))

pyplot.plot(history.history['loss'],label='train')
    #pyplot.plot(history.history['val_loss'],label='test')
pyplot.legend()
pyplot.show()

yhat = model.predict(test_x)
y_te = yhat.reshape(Ndays_te*11,)

rmse2 += mean_squared_error(y_te,t_te)*Ndays_te*11

rmse = sqrt(rmse2/4026)*1087.4396/2
print('Test RMSE: %.3F' % rmse)

In [ ]:
#Linear Regression
from sklearn.linear_model import LinearRegression

#Reshape data to hourly format
x_tr_hourly = x_tr.reshape(-1, 9)  # All hours, 9 features each
t_tr_hourly = t_tr.reshape(-1)     # Target values for each hour

x_te_hourly = x_te.reshape(-1, 9)
t_te_hourly = t_te.reshape(-1)

#Train Linear Regression on hourly data
lr_model = LinearRegression()
lr_model.fit(x_tr_hourly, t_tr_hourly)

#Predict and denormalize
lr_preds_hourly = lr_model.predict(x_te_hourly)
lr_preds_denorm = (lr_preds_hourly + 1) * 700
test_t_denorm = (t_te_hourly + 1) * 700

#Compute RMSE
rmse_lr = sqrt(mean_squared_error(test_t_denorm, lr_preds_denorm))
print(f'Linear Regression Test RMSE: {rmse_lr} W/m²')

In [ ]:
# Persistence
# Reshape into days
x_te_days = x_te.reshape(Ndays_te, 11, 9)
t_te_days = t_te.reshape(Ndays_te, 11)

# Create arrays to store predictions and actual values
all_predictions = []
all_actuals = []

# For each day except the first one
for day_idx in range(1, Ndays_te):
    # Use the previous day's values as predictions for the current day
    previous_day_values = t_te_days[day_idx-1, :]
    current_day_actuals = t_te_days[day_idx, :]

    all_predictions.append(previous_day_values)
    all_actuals.append(current_day_actuals)

# Convert to arrays
persistence_preds = np.array(all_predictions)
target_vals = np.array(all_actuals)

# Denormalize
max_irradiance = 700
persistence_preds_denorm = (persistence_preds + 1) * (max_irradiance/2)
target_vals_denorm = (target_vals + 1) * (max_irradiance/2)

# Compute RMSE
rmse_persistence = np.sqrt(np.mean((target_vals_denorm.flatten() - persistence_preds_denorm.flatten())**2))
print(f'Persistence Model RMSE: {rmse_persistence:.4f} W/m²')

In [ ]:
#BPNN
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

#Constants from the paper
GHI_MIN = 0
GHI_MAX = 1400  #Based on paper’s expected GHI range
normaliasation_scale = GHI_MAX / 2  #for RMSE rescaling like LSTM code

#Extract inputs and targets
x_tr = train_data[:, 0:9]
t_tr = train_data[:, -1]
x_va = validate_data[:, 0:9]
t_va = validate_data[:, -1]
x_te = test_data[:, 0:9]
t_te = test_data[:, -1]

#Normalize inputs to [-1, 1]
min_vals = x_tr.min(axis=0)
max_vals = x_tr.max(axis=0)

def normalize(x):
    return 2 * (x - min_vals) / (max_vals - min_vals) - 1

x_tr = normalize(x_tr)
x_va = normalize(x_va)
x_te = normalize(x_te)

#Reshape data into sequences of 11 hours
Ndays_tr = x_tr.shape[0] // 11
Ndays_va = x_va.shape[0] // 11
Ndays_te = x_te.shape[0] // 11

train_x = x_tr.reshape(Ndays_tr, 99)
validate_x = x_va.reshape(Ndays_va, 99)
test_x = x_te.reshape(Ndays_te, 99)

train_t = t_tr.reshape(Ndays_tr, 11)
validate_t = t_va.reshape(Ndays_va, 11)
test_t = t_te.reshape(Ndays_te, 11)

#BPNN Model
model = Sequential([
    Dense(50, activation='tanh', input_shape=(99,)),
    Dense(11, activation='linear')
])

optimizer = SGD(learning_rate=0.01, momentum=0.0)
model.compile(optimizer=optimizer, loss='mse')

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=50,
    restore_best_weights=True
)

#Train
history = model.fit(
    train_x, train_t,
    validation_data=(validate_x, validate_t),
    epochs=2500,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

#Denormalize predicted and true values
def denormalize(data):
    return (data + 1) * normaliasation_scale  # same as * 1400 / 2

predictions = model.predict(test_x)
predictions_denorm = denormalize(predictions)
test_t_denorm = denormalize(test_t)

#Calculate RMSE
rmse = sqrt(mean_squared_error(test_t_denorm.flatten(), predictions_denorm.flatten()))
print(f'\nBPNN Test RMSE: {rmse} W/m²')
print(f'Best epoch: {np.argmin(history.history["val_loss"]) + 1}')

#Plot Training History
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('BPNN Training History')
plt.legend()
plt.grid(True)
plt.show()